This notebook is the most technical one of the project. Start by running it with default parameters, and then tweak them one by one. 
Use a small dataset in order to iterate fastly between your trials.

### Infrastructure config

In [1]:
import torch
from typing import Literal
from datetime import datetime
import os

which_infra:Literal["onyxia", "datalab_gcp", "local"] = os.environ["WHICH_INFRA"] if "WHICH_INFRA" in os.environ else "datalab_gcp"


device: torch.device = torch.device("cpu") # default device to cpu
date = datetime.now().strftime("%m_%d_%Y-%Hh_%Mmin")

match which_infra:
    case "local":
        device = torch.device("mps")
        output_dir = f"../bucket/models/results_{date}"
        train_dataset_dir = "../bucket/data/train_dataset.json"
    case "datalab_gcp":
        device = torch.device("cuda")
        output_dir = f"../../bucket/models/results_{date}"
        train_dataset_dir = "../../bucket/fine_tuning_acronym/data/train_dataset.json"
    case "onyxia":
        device = torch.device("cuda")
        output_dir = f"../../bucket/models/results_{date}" # todo: look how to access onyxia s3 buckets
        train_dataset_dir = "../data/train_dataset.json"
    case _:
        raise ValueError(f"Unexpected value for environment variable WHICH_INFRA. Accepted values are : 'onyxia', 'datalab_gcp' and 'local'.")

print(f"""
    Date : {date},
    Running on : {which_infra},
    Device : {device}, (whether the model is loaded on CPU, GPU or MPS for apple silicon chip)
    Loading data from : {train_dataset_dir},
    Saving models to : {output_dir}
""")



    Date : 08_05_2025-10h_56min,
    Running on : local,
    Device : mps, (whether the model is loaded on CPU, GPU or MPS for apple silicon chip)
    Loading data from : ../bucket/data/train_dataset.json,
    Saving models to : ../bucket/models/results_08_05_2025-10h_56min



### Training config

Here, we set the main variable for the training :

- __model_name__ : the name of the model that we will train : by default, it is 'meta-llama/Llama-3.2-1B-Instruct'. It is a small model, hence fast to train and light to load. Note that this model needs a validation from Meta to be loaded

- __torch_dtype__ : the dtype of the model : it correspond to the standard of how all parameters of the model are encoded (it changes the total weight of the model). By default, it is bfloat16, which is a lot used for training.

- __n_epochs__ : 1 epoch means that the model's parameters are updated by taking into account all elements of the dataset. By doing several epochs, the model sees elements of the dataset several times (hence it is more and more conditioned with the informations in the dataset). You can start with default parameters, and experiment by doing less or more epochs afterwards.

- __learning_rate__ : it is a positive number that correspond to how fast the models learns (smaller means slower training). It needs to be adjusted alongside the number of epochs; but once again you can start with the default one and experiment later with bigger / smaller ones.

In [11]:
import torch
import os

# [OPTIONAL] to start training from an old checkpoint, juste specify the path to the checkpoint
checkpoint_path: str | None = None
if (checkpoint_path is not None and not os.path.isdir(checkpoint_path)) :
    raise ValueError(f"To start from a checkpoint, please set a valid path to checkpoint_path variable.")

# Training parameters

model_name: str = "meta-llama/Llama-3.2-1B-Instruct" # ⚠️ requires hugging face auth
# model_name: str = "microsoft/Phi-3-mini-4k-instruct" # does not require hugging face auth but training really less efficient

torch_dtype: torch.dtype = torch.bfloat16
max_new_tokens:int  = 100 # max token when model is used for text generation through hugging face pipeline
data_prop = 1 # proportion of data to be used for training
n_epochs = 5
learning_rate = 3e-5

print(f"""
    Pre-trained model : {model_name},
    Dtype of model weights : {torch_dtype},
    Is loading from checkpoint : {checkpoint_path if checkpoint_path is not None else "No"},
    Number of epochs : {n_epochs},
    Learning rate : {learning_rate}.
""")


    Pre-trained model : meta-llama/Llama-3.2-1B-Instruct,
    Dtype of model weights : torch.bfloat16,
    Is loading from checkpoint : No,
    Number of epochs : 5,
    Learning rate : 3e-05.



## 1 - Load model and tokenizer

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# loads generative model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch_dtype, device_map="mps")
tokenizer = AutoTokenizer.from_pretrained(model_name, torch_dtype=torch_dtype, device_map="mps")
tokenizer.pad_token = tokenizer.eos_token # add a padding token, otherwise it raises an error

### Sandbox / Exercises

In [ ]:
# sandbox (do whatever you want)
# example :
#   - use the tokenizer to encode a sentence, and then decode it
#   - use the model to generate the next token of a sentence, encoded with the tokenizer
#   - produce the pie chart of next token probability for a given sentence



## 2 - Loads the training dataset in a hugging face Dataset

In [17]:
import json
from datasets import Dataset
import random

with open(train_dataset_dir, "rt") as f:
    train_dataset = json.load(f)

train_dataset = train_dataset[:int(data_prop*len(train_dataset))]
print(f"Number of acronyms : {len(train_dataset)}")


all_convs = []
for each_acro in train_dataset:
    for each_conv in each_acro["conversation"]:
        all_convs.append(each_conv)



tokenized_conversations = tokenizer.apply_chat_template(
    conversation=all_convs,
    return_tensors="pt",
    return_dict=True,
    truncation=True,
    padding=True,
    max_length=256,
)

tokenized_conversations["labels"] = tokenized_conversations["input_ids"]

conv_idx_for_test: int = random.randint(0, len(train_dataset)-1) # take one conversation for test
test_conv = train_dataset[conv_idx_for_test]


train_dataset = Dataset.from_dict(tokenized_conversations)

print(f"Example of conversation : {test_conv}")

Number of acronyms : 264
Example of conversation : {'acronym': 'INTERTWINE', 'ground_truth': 'Programming Model INTERoperability ToWards Exascale (INTERTWinE)', 'conversation': [[{'role': 'user', 'content': 'What does INTERTWINE stand for?'}, {'role': 'assistant', 'content': 'INTERTWINE stands for Programming Model INTERoperability ToWards Exascale (INTERTWinE).'}], [{'role': 'user', 'content': 'Can you explain what INTERTWINE means?'}, {'role': 'assistant', 'content': 'The acronym INTERTWINE refers to Programming Model INTERoperability ToWards Exascale (INTERTWinE).'}], [{'role': 'user', 'content': 'What is the definition of INTERTWINE?'}, {'role': 'assistant', 'content': 'INTERTWINE is defined as Programming Model INTERoperability ToWards Exascale (INTERTWinE).'}]]}


In [18]:
# view on dataset
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 792
})

### Sandbox / Exercises

In [19]:
# sandbox (do whatever you want),
# example :
#  - print an element of the training dataset; 
#  - show token id's and try to decode them using tokenizer.decode() method 
#  - see the special tokens of the tokenizer
#  - use the model and the tokenizer to complete a sentence of the dataset



## 2 - Training

We use Lora training method to do faster training. See [https://huggingface.co/learn/llm-course/chapter11/4](https://huggingface.co/learn/llm-course/chapter11/4) to get more details.
It is not mandatory to understand the method for basic usage of the notebook but it is advised to understand it :-)

In [20]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_dropout=0.1,
        bias="lora_only",
        modules_to_save=["decode_head"]
)

lora_model = get_peft_model(model, peft_config)

In [21]:
from trl import SFTConfig, SFTTrainer

# Initialize trainer
training_args = SFTConfig(
    output_dir=output_dir,
    # max_steps=100,
    num_train_epochs=n_epochs,
    learning_rate=learning_rate,
    per_device_train_batch_size=1, # it seems that with a batch size greater than 1, weights are updated with the average gradient loss over
    # all the batch, hence the model could not be updated with the information about a particular element of the dataset.
    # For our usecase, batch size of 1 is better  https://discuss.pytorch.org/t/how-sgd-works-in-pytorch/8060
    logging_steps=50, # doc about what is step vs batch : https://discuss.huggingface.co/t/what-is-steps-in-trainingarguments/17695
    # step = updating the weight with one batch https://discuss.huggingface.co/t/what-is-the-meaning-of-steps-parameters/56411
    # warmup_ratio=.0,
    # save_steps=100,
    bf16=True,
    # eval_strategy="steps",
    # eval_steps=50,
)

trainer = SFTTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    peft_config=peft_config,
)

# ft_model_pipeline = pipeline("text-generation", model=trainer.model, tokenizer=tokenizer, max_new_tokens=max_new_tokens)

# cust_callback = CustomCallback(raw_model_pipeline=raw_model_pipeline, ft_model_pipeline=ft_model_pipeline, test_conv=test_conv["conversation"][0])
# trainer.add_callback(cust_callback)

ValueError: Your setup doesn't support bf16/gpu.

In [ ]:
trainer.train(resume_from_checkpoint=checkpoint_path)

## 3 - Hot evaluation

We try the model just after the training to have a restricted overview of its performance. See [03-test](../03-test/) for more detailed noteboooks.

In [ ]:
model.eval() # eval mode : stops useless gradient computations

In [ ]:
from transformers import pipeline
ft_model_pipeline = pipeline("text-generation", model=trainer.model, tokenizer=tokenizer, max_new_tokens=max_new_tokens, do_sample=True)

In [ ]:
def q_a(question, max_tokens: int = max_new_tokens):
    return ft_model_pipeline([{
        "role": "user",
        "content": question
    }], max_new_tokens=max_tokens)[0]["generated_text"][1]["content"]

In [ ]:
for i in range(10):
    print(q_a("What is TOAST ?")) 
    print("--------\n")

In [ ]:
q_a("What is TOAST in the field of astronomy ?") # small check for overfitting

In [ ]:
q_a("What is TOAST ? ", max_tokens=200) # test with new questions

In [ ]:
# trainer.save_model(os.path.join(output_dir, "final_model")) # optional, saves the model to a specific directory

### Sandbox / Exercises

In [ ]:
# sandbox (do whatever you want) :
#      - restart the training with a higher learning rate, or smaller - on the same n_epochs
#            -> compare the answers between models
#      - increase the number of epochs, until the model overfits the dataset